In [1]:
import numpy as np
import pandas as pd
from string import punctuation

In [2]:
!mkdir data

In [8]:
!mv '/content/data/отзывы за лето (1).xls' 'data/summer.xls'

In [11]:
!ls data

summer.xls


In [181]:
data = pd.read_excel(open('data/summer.xls', 'rb'))
data.head()

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,В целом удобноное приложение...из минусов хотя...,2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше ...,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14


In [182]:
data.shape

(20659, 3)

In [183]:
data.drop('Date', axis=1, inplace=True)

In [184]:
data.head()

,Rating,Content
0,5,It just works!
1,4,В целом удобноное приложение...из минусов хотя...
2,5,Отлично все
3,5,Стал зависать на 1% работы антивируса. Дальше ...
4,5,"Очень удобно, работает быстро."


In [185]:
max_words = 200
max_len = 150
num_classes = 1
epochs = 20
batch_size = 512
print_batch_n = 100

In [186]:
#!pip install stop-words
#!pip install pymorphy2

In [187]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [188]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub("\sне", "не", txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    return " ".join(txt)

In [189]:
data['Content'] = data['Content'].apply(preprocess_text)

In [190]:
data.head()

,Rating,Content
0,5,it just works
1,4,целое удобноной приложениеиз минус хотеть боль...
2,5,отлично
3,5,зависать 1 работа антивирус ранее пользоваться...
4,5,удобно работать быстро


In [191]:
from sklearn.model_selection import train_test_split

In [192]:
X = data.drop(columns = ['Rating']).copy()
y = data['Rating']

train_size=0.8
X_train, X_rem, y_train, y_rem = train_test_split(X,y, train_size=0.8)

test_size = 0.5
X_val, X_test, y_val, y_test = train_test_split(X_rem,y_rem, test_size=0.5)

In [193]:
train_corpus = " ".join(X_train["Content"])
train_corpus = train_corpus.lower()

In [194]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

tokens = word_tokenize(train_corpus)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [195]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [196]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [197]:
tokens_filtered_top[:8]

['приложение',
 'удобно',
 'работать',
 'удобный',
 'отлично',
 'нравиться',
 'хороший',
 'отличный']

In [198]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [199]:
def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [200]:
x_train = np.asarray([text_to_sequence(text, max_len) for text in X_train["Content"]], dtype=np.int32)
x_test = np.asarray([text_to_sequence(text, max_len) for text in X_test["Content"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in X_val["Content"]], dtype=np.int32)

In [201]:
x_train[8]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 1], dtype=int32)

In [202]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.callbacks import TensorBoard 
# from keras.objectives import categorical_crossentropy
from keras.callbacks import EarlyStopping  
import tensorflow as tf

from sklearn.feature_extraction.text import TfidfVectorizer

In [203]:
from keras_preprocessing.sequence import pad_sequences

In [204]:
y_train = pd.DataFrame(y_train)
y_test = pd.DataFrame(y_test)

In [205]:
pd.unique(data['Rating'])

array([5, 4, 2, 3, 1])

In [206]:
num_classes = 6
y_train = tf.keras.utils.to_categorical(y_train['Rating'], num_classes)
y_test = tf.keras.utils.to_categorical(y_test['Rating'], num_classes)

In [207]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

In [208]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [209]:
early_stopping=EarlyStopping(monitor='val_loss')  
history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[early_stopping])

Epoch 1/20
30/30 [==============================] - 18s 584ms/step - loss: 1.2859 - accuracy: 0.6802 - val_loss: 0.9454 - val_accuracy: 0.7072
Epoch 2/20
30/30 [==============================] - 15s 496ms/step - loss: 0.8528 - accuracy: 0.7149 - val_loss: 0.7562 - val_accuracy: 0.7526


In [210]:
score = model.evaluate(x_test, y_test, batch_size=batch_size, verbose=1)
print('\n')
print('Test loss:', score[0])
print('Test accuracy:', score[1])

5/5 [==============================] - 1s 101ms/step - loss: 0.7443 - accuracy: 0.7619


Test loss: 0.7443206310272217
Test accuracy: 0.7618586421012878


## 2.1. Модель со слоем tf.keras.layers.Embedding с предобученными векторами

In [211]:
df_w2v = data.copy()
df_w2v = df_w2v[df_w2v['Rating'] != 3]
df_w2v['target'] = (df_w2v['Rating'] > 3)*1
df_w2v = df_w2v.drop(['Rating'], axis=1)

In [212]:
df_w2v['target'] = df_w2v['target'].astype(int)
df_w2v['target'].value_counts()

1    16724
0     3024
Name: target, dtype: int64

In [213]:
df_train = df_w2v.loc[:4000]
df_val = df_w2v.loc[4001:]

In [214]:
df_train['Content'] = df_train['Content'].apply(preprocess_text)
df_val['Content'] = df_val['Content'].apply(preprocess_text)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [215]:
train_corpus = " ".join(df_train["Content"])
train_corpus = train_corpus.lower()

In [216]:
max_words = 200
max_len = 40

In [217]:
tokens = word_tokenize(train_corpus)
tokens_filtered = [word for word in tokens if word.isalnum()]
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [218]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]
x_train = np.asarray([text_to_sequence(text, max_len) for text in df_train["Content"]], dtype=np.int32)
x_val = np.asarray([text_to_sequence(text, max_len) for text in df_val["Content"]], dtype=np.int32)
x_train

array([[  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,  97,  98,  13],
       [  0,   0,   0, ...,   0,   0,   5],
       ...,
       [  0,   0,   0, ...,   0,   1, 186],
       [  0,   0,   0, ...,   0,   0,   0],
       [  0,   0,   0, ...,   0,   0,   0]], dtype=int32)

In [219]:
df_train["target"].unique()

array([1, 0])

In [220]:
from tensorflow.keras import utils as np_utils

In [221]:
num_classes = 2
y_train_w2v = np_utils.to_categorical(df_train["target"], num_classes)
y_val_w2v = np_utils.to_categorical(df_val["target"], num_classes)

In [222]:
from gensim.models import Word2Vec

In [223]:
modelW2V = Word2Vec(sentences=df_train['Content'].apply(str.split), size=40, window=5, min_count=1)

In [224]:
vect_idf = TfidfVectorizer()
vect_idf.fit_transform(df_train['Content'])
tfidf = dict(zip(vect_idf.get_feature_names(), vect_idf.idf_))

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [225]:
def get_vect_mean(txt):
    vector_w2v = np.zeros(40)
    n_w2v = 0
    for wrd in txt.split():
        if wrd in modelW2V:
            vector_w2v += modelW2V[wrd]
            n_w2v += 1
    if n_w2v > 0:
        vector_w2v = vector_w2v / n_w2v
    return vector_w2v

In [226]:
from tqdm import tqdm_notebook

arr_vect = []
for txt in tqdm_notebook(df_train['Content']):
    arr_vect.append(get_vect_mean(txt))
    
arr_vect_valid = []
for txt in tqdm_notebook(df_val['Content']):
    arr_vect_valid.append(get_vect_mean(txt))
    
x_train_w2v = np.asarray(arr_vect)    
x_val_w2v = np.asarray(arr_vect_valid)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/3825 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


  0%|          | 0/15923 [00:00<?, ?it/s]

In [227]:
from tensorflow.keras.layers import Dense, Embedding, GlobalMaxPooling1D, Conv1D, Activation

In [228]:
model_w2v = tf.keras.Sequential()
model_w2v.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model_w2v.add(Conv1D(128, 3))
model_w2v.add(Activation("relu"))
model_w2v.add(GlobalMaxPooling1D())
model_w2v.add(Dense(10))
model_w2v.add(Activation("relu"))
model_w2v.add(Dense(num_classes))
model_w2v.add(Activation('softmax'))

model_w2v.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, 40, 128)           25600     
                                                                 
 conv1d_5 (Conv1D)           (None, 38, 128)           49280     
                                                                 
 activation_15 (Activation)  (None, 38, 128)           0         
                                                                 
 global_max_pooling1d_5 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_10 (Dense)            (None, 10)                1290      
                                                                 
 activation_16 (Activation)  (None, 10)                0         
                                                      

In [229]:
optimizer = tf.keras.optimizers.Adam(lr=0.0001)

model_w2v.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['AUC'])

tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir='logs/my_model_plus_w2v', 
    write_graph=False, update_freq=100, profile_batch=0)

/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [230]:
NUM_EPOCHS = 8

In [231]:
history = model_w2v.fit(
    x_train_w2v, y_train_w2v,
    epochs=NUM_EPOCHS,
    validation_split=0.1,
 )

Epoch 1/8
108/108 [==============================] - 3s 18ms/step - loss: 0.5919 - auc: 0.8808 - val_loss: 0.5024 - val_auc: 0.8851
Epoch 2/8
108/108 [==============================] - 2s 15ms/step - loss: 0.4373 - auc: 0.8762 - val_loss: 0.3790 - val_auc: 0.8851
Epoch 3/8
108/108 [==============================] - 2s 15ms/step - loss: 0.3747 - auc: 0.8733 - val_loss: 0.3585 - val_auc: 0.8851
Epoch 4/8
108/108 [==============================] - 2s 15ms/step - loss: 0.3685 - auc: 0.8782 - val_loss: 0.3568 - val_auc: 0.8851
Epoch 5/8
108/108 [==============================] - 2s 15ms/step - loss: 0.3682 - auc: 0.8776 - val_loss: 0.3567 - val_auc: 0.8851
Epoch 6/8
108/108 [==============================] - 2s 15ms/step - loss: 0.3682 - auc: 0.8772 - val_loss: 0.3567 - val_auc: 0.8851
Epoch 7/8
108/108 [==============================] - 2s 15ms/step - loss: 0.3682 - auc: 0.8792 - val_loss: 0.3568 - val_auc: 0.8851
Epoch 8/8
108/108 [==============================] - 2s 15ms/step - loss: 0.

In [232]:
loss, accuracy = model_w2v.evaluate(x_train_w2v, y_train_w2v, batch_size=batch_size, verbose=False)
print("Training Loss:  {:.4f}".format(loss))
print("Training Accuracy:  {:.4f}".format(accuracy))
print('\n')
loss, accuracy = model_w2v.evaluate(x_val_w2v, y_val_w2v, batch_size=batch_size, verbose=False)
print("Testing Loss:  {:.4f}".format(loss))
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Loss:  0.3669
Training Accuracy:  0.8800


Testing Loss:  0.4496
Testing Accuracy:  0.8389


## 2.2 Модель со слоем tf.keras.layers.Embedding по умолчанию

In [233]:
exclude = set(punctuation)
sw = set(get_stop_words("ru"))
morpher = MorphAnalyzer()

def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in exclude)
    txt = txt.lower()

    txt = re.sub("\sне", "не", txt)
    
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    txt = [word for word in txt if len(word)>1]
    
    return " ".join(txt)

data['text'] = data['Content'].apply(preprocess_text)

In [234]:
data = data[data['Rating'] != 3]
data['target'] = (data['Rating'] > 3)*1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [235]:
df = data.drop(['Content', 'Rating'], axis=1)
print(df.iloc[0])

text      it just works
target                1
Name: 0, dtype: object


In [236]:
df_train, df_val = train_test_split(df, test_size=0.2,random_state=8)

In [237]:
text_corpus_train = df_train['text'].values
text_corpus_val = df_val['text'].values

In [238]:
tokenizer = Tokenizer(num_words=None, 
                     filters='#$%&()*+-<=>@[\\]^_`{|}~\t\n',
                     lower = False, split = ' ')
tokenizer.fit_on_texts(text_corpus_train)

sequences_train = tokenizer.texts_to_sequences(text_corpus_train)
sequences_val = tokenizer.texts_to_sequences(text_corpus_val)

word_count = len(tokenizer.index_word) + 1
training_length = max([len(i.split()) for i in text_corpus_train])

x_train = pad_sequences(sequences_train, maxlen=training_length)
x_val = pad_sequences(sequences_val, maxlen=training_length)

In [239]:
num_classes = 2
y_train = np_utils.to_categorical(df_train["target"], num_classes)
y_val = np_utils.to_categorical(df_val["target"], num_classes)

In [240]:
model = tf.keras.Sequential()
model.add(Embedding(input_dim=word_count, 
                    output_dim=128, 
                    input_length=training_length))
model.add(Conv1D(128, 3))
model.add(Activation("relu"))
model.add(GlobalMaxPooling1D())
model.add(Dense(10))
model.add(Activation("relu"))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_6 (Embedding)     (None, 100, 128)          1367552   
                                                                 
 conv1d_6 (Conv1D)           (None, 98, 128)           49280     
                                                                 
 activation_18 (Activation)  (None, 98, 128)           0         
                                                                 
 global_max_pooling1d_6 (Glo  (None, 128)              0         
 balMaxPooling1D)                                                
                                                                 
 dense_12 (Dense)            (None, 10)                1290      
                                                                 
 activation_19 (Activation)  (None, 10)                0         
                                                      

In [241]:
batch_size = 1024

In [242]:
optimizer = tf.keras.optimizers.Adam(lr=0.0001)

model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [243]:
history = model.fit(
    x_train, y_train,
    epochs=NUM_EPOCHS,
    validation_data=(x_val,y_val),
    )

Epoch 1/8
494/494 [==============================] - 22s 42ms/step - loss: 0.3620 - accuracy: 0.8482 - val_loss: 0.2586 - val_accuracy: 0.8714
Epoch 2/8
494/494 [==============================] - 21s 42ms/step - loss: 0.2065 - accuracy: 0.9128 - val_loss: 0.1961 - val_accuracy: 0.9122
Epoch 3/8
494/494 [==============================] - 20s 41ms/step - loss: 0.1586 - accuracy: 0.9371 - val_loss: 0.1766 - val_accuracy: 0.9228
Epoch 4/8
494/494 [==============================] - 20s 41ms/step - loss: 0.1315 - accuracy: 0.9514 - val_loss: 0.1731 - val_accuracy: 0.9235
Epoch 5/8
494/494 [==============================] - 20s 40ms/step - loss: 0.1093 - accuracy: 0.9618 - val_loss: 0.1744 - val_accuracy: 0.9268
Epoch 6/8
494/494 [==============================] - 20s 40ms/step - loss: 0.0908 - accuracy: 0.9699 - val_loss: 0.1818 - val_accuracy: 0.9266
Epoch 7/8
494/494 [==============================] - 20s 41ms/step - loss: 0.0752 - accuracy: 0.9763 - val_loss: 0.1918 - val_accuracy: 0.9233

In [244]:
loss, accuracy = model.evaluate(x_train, y_train, batch_size=batch_size, verbose=False)
print("Training Loss:  {:.4f}".format(loss))
print("Training Accuracy:  {:.4f}".format(accuracy))
print('\n')
loss, accuracy = model.evaluate(x_val, y_val, batch_size=batch_size, verbose=False)
print("Testing Loss:  {:.4f}".format(loss))
print("Testing Accuracy:  {:.4f}".format(accuracy))

Training Loss:  0.0532
Training Accuracy:  0.9853


Testing Loss:  0.1915
Testing Accuracy:  0.9253


###  Эмбединг показал лучшие результаты чем wors2vec